In [ ]:
# Import
import os
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
from matplotlib.axes import Axes
from matplotlib.figure import Figure
import matplotlib.font_manager as font_manager
import matplotlib.patches as mpatches
from info import *

# Import local_features and util (hack)
import sys
import os
sys.path.append(os.path.abspath(".."))
from local_features import *
from util import *


INFO = Info()


In [ ]:
# Set Heuristica as font

font_path = os.path.join(os.path.abspath('..'), 'font.otf')
assert os.path.exists(font_path)
font_manager.fontManager.addfont(font_path)
prop = font_manager.FontProperties(fname=font_path)

plt.rcParams['font.family'] = prop.get_name()
plt.rcParams['font.size'] = 12

In [ ]:
# Piano roll for a specified performance
from recording import OutputName

participant_id: int = 2
performance_id: int = 0

performance: Performance = INFO.data(participant_id, performance_id)
print(ALGORITHM_LONG_NAMES[performance.algorithm])
recording: Recording = performance.recording
midi_notes: list[MidiNote] = all_notes(recording.measures)

colors = ['#DBD300', '#008FEF']

fig: Figure
ax: Axes
fig, ax = plt.subplots(figsize=(20, 5))

# Plot each note with the corresponding color
for note in midi_notes:
    color = colors[0] if note.output_name == OutputName.LOOPBACK else colors[1]
    ax.broken_barh([(note.time, note.length)], (note.note - 0.4, 0.8), facecolors=color)

# Formatting
ax.set_title(f"Piano roll of a performance (Billie's Bounce with {ALGORITHM_LONG_NAMES[performance.algorithm]})")
ax.set_xticks(np.arange(0, 48+4, 4))
ax.set_yticks([])

ax.grid(True)

# Add legend
legend_patches = [
    plt.Line2D([0], [0], color=colors[0], lw=4, label="Pupil"),
    plt.Line2D([0], [0], color=colors[1], lw=4, label="Algorithm")
]
ax.legend(handles=legend_patches, loc="upper right")

plt.savefig('charts/0_piano_roll.svg')

In [ ]:
# Ratings per algorithm (side-to-side bars)

fig, axs = plt.subplots(NUM_QUESTIONS, figsize=(8, 10))

colors = ['#B45C1F', '#DE8B52', '#1FB42A', "#52DE5C"]

questions = [
    "I feel that the algorithm could respond well to me",
    "I feel that I could respond well to the algorithm",
    "I feel that the algortihm gives me inspiration",
    "I enjoyed playing this song with the algorithm",
]

width = 0.166

for algorithm_index in range(NUM_ALGORITHMS):
    # Get scores of performances of algorithm
    performances = [
        INFO.info[id]
        for id in INFO.info
        if INFO.info[id].algorithm == algorithm_index 
    ]

    print(performances[0].scores)

    for q in range(NUM_QUESTIONS):
        q_scores: list[int] = [sum([p.scores[p.participant_id][q] == v for p in performances]) for v in range(1, 6)]
        axs[q].title.set_text(f'"{questions[q]}"')
        axs[q].bar(x=np.array(range(1, 6)) + width * algorithm_index - width*1.5,height=q_scores, width=width, color=colors[algorithm_index])
        axs[q].set_yticks(range(0, 10, 5))

legend_patches = [
    mpatches.Patch(color=colors[algorithm_index], label=ALGORITHM_SHORT_NAMES[algorithm_index]) for algorithm_index in range(NUM_ALGORITHMS)
]
fig.legend(handles=legend_patches, loc="outside upper right")

plt.savefig('charts/response_scores.png')

In [ ]:
# Enjoyability per algorithm (violin plot)

# Evaluate this specified question and algorithm
question_index: int = 3

colors = ['#B45C1F', '#DE8B52', '#1FB42A', "#52DE5C"]

all_reports: list[list[int]] = []

algorithm_index: int
for algorithm_index in range(NUM_ALGORITHMS):
    # Get performances of algorithm
    performances = [
        INFO.info[id]
        for id in INFO.info
        if INFO.info[id].algorithm == algorithm_index 
    ]

    # Get self-reports of performances of algorithm
    reports: list[int] = [
        performance.scores[performance.participant_id][question_index]
        for performance in performances
    ]

    all_reports.append(reports)

fig: Figure
ax: Axes
fig, ax = plt.subplots()
plots = ax.violinplot(all_reports, showmeans=True)
ax.set_title('"I enjoyed playing this song with the algorithm", per algorithm')

# Set colors
for pc, color in zip(plots['bodies'], colors):
    pc.set_facecolor(color)
plots['cmeans'].set_colors(colors)
plots['cmins'].set_colors(colors)
plots['cmaxes'].set_colors(colors)
plots['cbars'].set_colors(colors)

ax.yaxis.grid(True)
ax.set_ylim([0.8, 5.2])
_ = ax.set_xticks([y + 1 for y in range(NUM_ALGORITHMS)], labels=[ALGORITHM_SHORT_NAMES[i] for i in range(NUM_ALGORITHMS)])
ax.set_yticks(range(1, 6))

ax.set_ylabel("Likert score")

print([np.average(s) for s in all_reports])

plt.savefig('charts/2_algorithm_enjoyability.svg')

In [ ]:
# Reciprocity scores

# Evaluate this specified question and algorithm


colors = ['#B45C1F', '#DE8B52', '#1FB42A', "#52DE5C"]

all_reports: list[list[int]] = []

question_index: int
for question_index in [0, 1]:
    for algorithm_index in range(NUM_ALGORITHMS):
        # Get performances of algorithm
        performances: list[Performance] = [
            INFO.info[id]
            for id in INFO.info
            if INFO.info[id].algorithm == algorithm_index 
        ]

        # Get reports of performances of algorithm
        reports: list[int] = [
            performance.scores[performance.participant_id][question_index]
            for performance in performances
        ]

        all_reports.append(reports)

fig: Figure
ax: Axes
fig, ax = plt.subplots(figsize=(10, 5))
plots = ax.violinplot(all_reports, showmeans=True)
ax.set_title('Reciprocity ratings, per algorithm')

# Set colors
for pc, color in zip(plots['bodies'], colors * 2):
    pc.set_facecolor(color)
plots['cmeans'].set_colors(colors)
plots['cmins'].set_colors(colors)
plots['cmaxes'].set_colors(colors)
plots['cbars'].set_colors(colors)

ax.yaxis.grid(True)
ax.set_ylim([0.8, 5.2])
labels = ALGORITHM_SHORT_NAMES * 2
labels[1] += "\nAlgorithm-to-pupil"
labels[5] += "\nPupil-to-algorithm"

_ = ax.set_xticks([y + 1 for y in range(NUM_ALGORITHMS * 2)], labels=labels)
ax.set_yticks(range(1, 6))

ax.set_ylabel("Likert score")

print([np.average(s) for s in all_reports])

plt.savefig('charts/8_algorithm_reciprocity.svg')

In [ ]:
# Metrics over fours (dot plot)


fig, axs = plt.subplots(1, 4, figsize=(24, 4))
fig.subplots_adjust(hspace=0.5)

# Get metrics over (the one) canonical solos
# SONG_NAMES = ["Billies Bounce"]
# NUM_SONGS = 1
# for song_index in range(NUM_SONGS):
#     file_name: str = os.path.join(os.getenv('APPDATA'), "Godot\\app_userdata\\Program\\saves", SONG_NAMES[song_index], "solo.notes")

#     recording = Recording(file_name, num_measures=48)

#     recording_metrics: list[float] = []
#     for four in recording.fours[:48//4]:
#         recording_metrics.append(note_share_135(four, [5, 9, 0]))
    
#     for ax in axs:
#         ax.plot(range(48//4), recording_metrics)

colors = ['#DBD300', '#008FEF']

width = 0.166

for algorithm_index in range(NUM_ALGORITHMS):
    # Get performances of algorithm
    performances = [
        INFO.info[id]
        for id in INFO.info
        if INFO.info[id].algorithm == algorithm_index 
    ]

    metrics = make_average_list(48 // 4)

    # Get metrics of all fours
    for performance in performances:
        for i, four in enumerate(performance.recording.fours):
            if i >= (48 // 4): continue

            bpm = 158
            pitch_classes = [5, 9, 0]
            
            # m = extrema_ratio(four, bpm)
            # ylabel = "Extrema ratio"
            # m = note_share_135(four, pitch_classes)
            # ylabel = "Root/third/fifth note share"
            # ticks = list(range(2))

            m = npvi(four, bpm)
            ticks = list(range(0, 101, 20))
            ylabel = "NPVI"

            metrics[i].add(m)

    avg_metrics = get_average(metrics)

    x = list(range(len(metrics)))
    c = [colors[i % 2] for i in x]

    ax = axs[algorithm_index]#[algorithm_index // 2][algorithm_index % 2]

    ax.set_title(ALGORITHM_LONG_NAMES[algorithm_index])
    ax.set_xlabel("Time (in measures)")
    if ylabel == "NPVI": ax.set_ylim(0, 100)
    if algorithm_index == 0: ax.set_ylabel(ylabel)

    ax.scatter(x, avg_metrics, c=c, marker='o')
    ax.set_yticks(ticks)

legend_patches = [
    mpatches.Patch(color=colors[i], label=["Human four", "Algorithm four"][i]) for i in range(2)
]
fig.legend(handles=legend_patches, loc="outside upper right")

plt.savefig('charts/5_metrics_npvi.svg')

In [ ]:
# Edit distances per algorithm

colors = ['#B45C1F', '#DE8B52', '#1FB42A', "#52DE5C"]

fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.set_title('Token edit distances per algorithm')

all_averages = [0] * NUM_ALGORITHMS

for algorithm_index in range(NUM_ALGORITHMS):
    # Get performances of algorithm
    performances = [
        INFO.info[id]
        for id in INFO.info
        if INFO.info[id].algorithm == algorithm_index 
    ]

    # Get average edit score for all performances of this algorithm
    avg = Avg()
    for performance in performances:
        avg.add(np.average(performance.edit_distances))
    average_edit_distance = avg.avg()

    all_averages[algorithm_index] = average_edit_distance


ax.set_ylabel('Edit distance')
ax.bar(ALGORITHM_SHORT_NAMES, all_averages, color=colors)

plt.savefig('charts/6_algorithm_edit_distances.svg')

In [ ]:
# Self / peer correlation

self_scores = []
peer_scores = []

for id in INFO.info:
    performance = INFO.info[id]

    p_self_scores = performance.scores[performance.participant_id]

    others = [performance.scores[par] for par in range(NUM_PARTICIPANTS) if par != performance.participant_id]
    p_peer_scores = [float(np.average(z)) for z in zip(others[0], others[1])]

    self_scores += p_self_scores
    peer_scores += p_peer_scores



fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.set_title('Self-rating versus peer rating')


# (via https://python-graph-gallery.com/scatterplot-with-regression-fit-in-matplotlib/)
# Fit linear regression via least squares with numpy.polyfit
# It returns an slope (b) and intercept (a)
# deg=1 means linear fit (i.e. polynomial of degree 1)
b, a = np.polyfit(self_scores, peer_scores, deg=1)

# Create sequence of 100 numbers from 0 to 100
xseq = np.linspace(0.7, 5.3, num=100)

# Plot regression line
ax.plot(xseq, a + b * xseq, color="#7777FF", lw=2.5)



ax.set_xlabel("Self-rating (Likert score)")
ax.set_ylabel("Peer rating (Likert score)")

ax.set_xlim((0.5, 5.5))
ax.set_ylim((0.5, 5.5))

ax.set_xticks(range(1, 6))
ax.set_yticks(range(1, 6))

ax.scatter(self_scores, peer_scores, color="#0000ff44")

plt.savefig('charts/7_rating_scatter.svg')

In [ ]:
# How does every four relate to the next, on average?

import matplotlib.patches as mpatches

colors = ['#B45C1F', '#DE8B52', '#1FB42A', "#52DE5C"]
width = 0.166

fig: Figure
ax: Axes
fig, ax = plt.subplots(layout='constrained')

ax.title.set_text("Token edit distance per algorithm")
ax.set_xticklabels(["algorithm-to-pupil", "pupil-to-algorithm", "average"])
ax.set_xticks(np.array(range(3)) + 0.25)
# ax.set_yticks(range(0, 120, 20))

algorithm_index: int
for algorithm_index in range(NUM_ALGORITHMS):
    print(f"algorithm {algorithm_index}")

    # Get all performances
    performances = [
        INFO.info[id]
        for id in INFO.info
        if INFO.info[id].algorithm == algorithm_index 
    ]

    edit_distances = np.zeros(48 // 4 - 1)

    for performance in performances:
        edit_distances += performance.edit_distances

    edit_distances /= len(performances)

    height = [
        np.average(edit_distances[::2]), # Algorithm to pupil
        np.average(edit_distances[1::2]), # Pupil to algorithm
        np.average(edit_distances), # Both ways
    ]

    stds = [
        np.std(edit_distances[::2]), # Algorithm to pupil
        np.std(edit_distances[1::2]), # Pupil to algorithm
        np.std(edit_distances), # Both ways
    ]

    x = np.array(range(3)) + width * algorithm_index
    ax.bar(x=x,height=height, width=width, color=colors[algorithm_index])

    plt.errorbar(x, height, stds, fmt='.', color='Black', elinewidth=2,errorevery=1, alpha=0.5, capsize = 2)

# Add legend
legend_patches = [
    mpatches.Patch(color=colors[algorithm_index], label=ALGORITHM_LONG_NAMES[algorithm_index]) for algorithm_index in range(NUM_ALGORITHMS)
]
fig.legend(handles=legend_patches, loc="outside upper right")

plt.savefig('charts/6_algorithm_edit_distances.svg')